In [1]:
import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import os


C:\Wesley_Tao\6.Software\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [23]:
import pandas as pd

In [2]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [3]:
myfile=[]
for file in os.listdir("../data/News/US"):
    myfile.append("../data/News/US/"+file)

In [4]:
article_data=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        article_data.append(thisdoc)

In [5]:
len(article_data)

25

In [6]:
doc_clean = [clean(doc).split() for doc in article_data] 

In [7]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

### FOX

In [12]:
myfile=[]
for file in os.listdir("../data/News_Original/FOX"):
    myfile.append("../data/News_Original/FOX/"+file)

In [13]:
myfile

['../data/News_Original/FOX/FOX.txt']

In [14]:
doc_complete=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        doc_complete.append(thisdoc)

In [37]:
from nltk.tokenize import word_tokenize
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
#     print(tokens_re.search(tokens[0]))
    if lowercase:
        tokens =  [token if emoticon_re.search(token)  else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'

In [38]:
from nltk.corpus import stopwords
import string
def remove_stop_words(tweet):
    out=tweet
    if(tweet):
        p=re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
        p2=re.compile(r'(?:@[\w_]+)')
        punctuation = list(string.punctuation)
        stop = stopwords.words('english') + punctuation + ['via',"\r",r'<[^>]+>',"RT","…","’","—","u"]+[r'(?:@[\w_]+)']
    #     print(stop)
        terms_stop = [term for term in preprocess(tweet) if p.match(term)==None and term not in stop and p2.match(term)==None]
    #     terms_stop = [term for term in preprocess(tweet) if term not in stop]
        out=" ".join(terms_stop)
    return(out)

In [42]:
doc_clean = [remove_stop_words(doc).split() for doc in doc_complete] 

In [43]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [44]:
Lda = gensim.models.ldamodel.LdaModel

In [45]:
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [19]:
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.003*"china" + 0.003*"farmer" + 0.003*"trade" + 0.003*"trump" + 0.003*"u" + 0.003*"agriculture" + 0.003*"said" + 0.003*"fox" + 0.003*"april" + 0.003*"house"'), (1, '0.023*"farmer" + 0.016*"trade" + 0.016*"china" + 0.012*"trump" + 0.012*"u" + 0.010*"agriculture" + 0.010*"said" + 0.008*"war" + 0.008*"fox" + 0.008*"house"'), (2, '0.003*"farmer" + 0.003*"trade" + 0.003*"u" + 0.003*"china" + 0.003*"said" + 0.003*"told" + 0.003*"trans" + 0.003*"brunt" + 0.003*"heavily" + 0.003*"oak"')]


In [54]:
def topic(list_document,num_topics,num_words):
    """list_document is a list of string , each element is a document"""
    num_topics=num_topics
    doc_clean = [remove_stop_words(doc).split() for doc in list_document] 
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    mylist=[]
    for topic in range(num_topics):
        termlist=ldamodel.print_topics(num_topics=num_topics, num_words=num_words)[topic][1].split("+")
        mylist.append(list(map(lambda term:term.split("*"),termlist)))
    pandas_list=[]
    for topic_index in range(num_topics):
        this_topic=pd.DataFrame(mylist[topic_index],columns=["prob","key_words"])
        this_topic["topic"]=topic_index
        pandas_list.append(this_topic)
    LDA_result=pd.concat(pandas_list)
    return(LDA_result)

In [55]:
topic(doc_complete,3,3)

,prob,key_words,topic
0,0.032,"""China""",0
1,0.028,"""US""",0
2,0.018,"""said""",0
0,0.006,"""farmers""",1
1,0.005,"""uncertainty""",1
2,0.004,"""corn""",1
0,0.020,"""said""",2
1,0.017,"""trade""",2
2,0.016,"""China""",2


### China Daily

In [71]:
media="ChinaDaily"

myfile=[]
for file in os.listdir("../data/News_Original/"+media):
    myfile.append("../data/News_Original/"+media+"/"+file)
doc_complete=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        doc_complete.append(thisdoc)


In [72]:
topic(doc_complete,5,5)

,prob,key_words,topic
0,0.029,"""US""",0
1,0.026,"""China""",0
2,0.018,"""said""",0
3,0.016,"""trade""",0
4,0.010,"""tariff""",0
0,0.027,"""said""",1
1,0.022,"""trade""",1
2,0.014,"""The""",1
3,0.014,"""China""",1
4,0.012,"""Graham""",1


In [26]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/ChinaDaily/10.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.031*"china" + 0.027*"u" + 0.023*"said" + 0.020*"trade" + 0.009*"chinese" + 0.009*"tariff" + 0.009*"soybean" + 0.007*"world" + 0.006*"lng" + 0.006*"would"'), (1, '0.001*"china" + 0.001*"u" + 0.001*"said" + 0.001*"trade" + 0.001*"soybean" + 0.001*"tariff" + 0.001*"world" + 0.001*"chinese" + 0.001*"would" + 0.001*"farmer"'), (2, '0.001*"china" + 0.001*"u" + 0.001*"said" + 0.001*"trade" + 0.001*"chinese" + 0.001*"world" + 0.001*"tariff" + 0.001*"soybean" + 0.001*"year" + 0.001*"lng"')]


### CNN

In [69]:
media="CNN"

myfile=[]
for file in os.listdir("../data/News_Original/"+media):
    myfile.append("../data/News_Original/"+media+"/"+file)
doc_complete=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        doc_complete.append(thisdoc)
topic(doc_complete,5,5)

,prob,key_words,topic
0,0.016,"""trade""",0
1,0.014,"""G""",0
2,0.013,"""5""",0
3,0.011,"""Trump""",0
4,0.011,"""China""",0
0,0.014,"""China""",1
1,0.014,"""trade""",1
2,0.013,"""US""",1
3,0.010,"""Chinese""",1
4,0.009,"""would""",1


In [28]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/CNN/CNNALL.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.001*"trade" + 0.001*"china" + 0.001*"said" + 0.001*"u" + 0.001*"trump" + 0.001*"state" + 0.001*"tariff" + 0.001*"united" + 0.001*"chinese" + 0.001*"war"'), (1, '0.001*"china" + 0.001*"trade" + 0.001*"said" + 0.001*"u" + 0.001*"trump" + 0.001*"war" + 0.001*"state" + 0.001*"united" + 0.001*"tariff" + 0.001*"chinese"'), (2, '0.022*"china" + 0.022*"trade" + 0.013*"u" + 0.013*"said" + 0.011*"trump" + 0.010*"chinese" + 0.009*"state" + 0.009*"war" + 0.009*"tariff" + 0.008*"united"')]


### Renmin

In [29]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/Renmin/Renmin.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.025*"china" + 0.018*"u" + 0.016*"trade" + 0.012*"chinese" + 0.009*"said" + 0.007*"world" + 0.007*"people" + 0.006*"product" + 0.006*"measure" + 0.006*"international"'), (1, '0.001*"china" + 0.001*"u" + 0.001*"trade" + 0.001*"chinese" + 0.001*"world" + 0.001*"said" + 0.001*"people" + 0.001*"product" + 0.001*"war" + 0.001*"international"'), (2, '0.001*"china" + 0.001*"u" + 0.001*"trade" + 0.001*"said" + 0.001*"chinese" + 0.001*"people" + 0.001*"world" + 0.001*"china’s" + 0.001*"product" + 0.001*"international"')]


In [67]:
media="Renmin"

myfile=[]
for file in os.listdir("../data/News_Original/"+media):
    myfile.append("../data/News_Original/"+media+"/"+file)
doc_complete=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        doc_complete.append(thisdoc)
topic(doc_complete,6,5)

,prob,key_words,topic
0,0.033,"""US""",0
1,0.026,"""China""",0
2,0.015,"""trade""",0
3,0.013,"""said""",0
4,0.010,"""The""",0
0,0.031,"""China""",1
1,0.024,"""trade""",1
2,0.009,"""said""",1
3,0.009,"""The""",1
4,0.008,"""new""",1


### SCMP

In [68]:
media="South China Morning Post"

myfile=[]
for file in os.listdir("../data/News_Original/"+media):
    myfile.append("../data/News_Original/"+media+"/"+file)
doc_complete=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        thisdoc=f.read()
        doc_complete.append(thisdoc)
topic(doc_complete,5,5)

,prob,key_words,topic
0,0.000,"""China""",0
1,0.000,"""US""",0
2,0.000,"""trade""",0
3,0.000,"""”""",0
4,0.000,"""Trump""",0
0,0.027,"""China""",1
1,0.024,"""US""",1
2,0.020,"""trade""",1
3,0.013,"""Trump""",1
4,0.011,"""war""",1


In [31]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/SCMP/SCMP.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.025*"china" + 0.024*"u" + 0.017*"chinese" + 0.016*"trade" + 0.015*"said" + 0.013*"2018" + 0.009*"canada" + 0.009*"photo" + 0.008*"tariff" + 0.008*"import"'), (1, '0.003*"u" + 0.003*"chinese" + 0.003*"china" + 0.003*"said" + 0.003*"trade" + 0.003*"official" + 0.003*"edition" + 0.003*"unilateral" + 0.003*"hope" + 0.003*"unfair"'), (2, '0.003*"china" + 0.003*"u" + 0.003*"chinese" + 0.003*"trade" + 0.003*"2018" + 0.003*"said" + 0.003*"photo" + 0.003*"apr" + 0.003*"war" + 0.003*"canada"')]


### SCM

In [34]:
fh = open("/Users/jiangyiran/Desktop/Stat_2nd/BDA/Project/News/SOUTHCHINAMORNING.txt")
doc1 = fh.read()
doc_complete = [doc1]
doc_clean = [clean(doc).split() for doc in doc_complete] 
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.002*"u" + 0.002*"trade" + 0.002*"china" + 0.002*"bank" + 0.002*"company" + 0.002*"bloomberg" + 0.002*"three" + 0.002*"war" + 0.002*"2018" + 0.002*"tariff"'), (1, '0.020*"china" + 0.020*"trade" + 0.018*"u" + 0.012*"bloomberg" + 0.011*"bank" + 0.010*"company" + 0.010*"new" + 0.008*"would" + 0.008*"war" + 0.008*"chinese"'), (2, '0.002*"trade" + 0.002*"china" + 0.002*"u" + 0.002*"new" + 0.002*"bank" + 0.002*"bloomberg" + 0.002*"would" + 0.002*"chinese" + 0.002*"trump’s" + 0.002*"tariff"')]


In [73]:
from __future__ import print_function, unicode_literals, division
from bz2 import BZ2File
import time
import plac
import ujson
from spacy.matcher import PhraseMatcher
import spacy
import time 
import gc
from spacy import displacy
import numpy as np
import pandas as pd
import os

In [74]:
nlp=spacy.load("en")
class FileProcessor():
   
    
    ''' we need pandas,spacy to function'''
    def __init__(self,name,text):
        self.docname=name
        self.doc=nlp(text)
        self.wordnum=len(self.doc)
        self.trivnum=len(self.doc)-len(self.SelectQualified())
        
    def GetTokenDF(self):
        """we want to get the whole dataframe with token details with the help of pandas """
        tokendf=pd.DataFrame(columns=["text","lemma","pos","tag","dep","is_stop"])
        for token in self.doc:
            tokendf.loc[token.text]=[token.text, token.lemma_ ,token.pos_, token.tag_, token.dep_, token.is_stop]
        return tokendf
    
    def SelectQualified(self):
        """this is to select qualified words display in dataframe"""
        tokendf=self.GetTokenDF()
        sampleset=["PROPN","NOUN","VERB","ADJ","ADV"] # pick only noun verb adj and adv words
        Qtoken=tokendf.loc[(tokendf.pos.isin(sampleset))& (tokendf.is_stop == False)] # is_stop  means the trivial words "of the a" 
        return Qtoken
    
    def GetEntityDF(self):
        """get entity data frame excluding cardinal and ordinal"""
        entitydf=pd.DataFrame(columns="text,start_char,end_char,label".split(","))
        for ent in self.doc.ents:
            entitydf.loc[ent.text]=[ent.text, ent.start_char, ent.end_char, ent.label_]
            entitydf=entitydf.loc[(entitydf.label!="CARDINAL") & (entitydf.label!="ORDINAL")] # cardinal and ordinal are not what we want
        return entitydf
    
    def GetWordChunkDF(self):
        """get word chunk  dataframe"""
        samplechunk=pd.DataFrame(columns="text, root.text, root.dep_,head.text".split(","))
        for chunk in self.doc.noun_chunks:
            samplechunk.loc[chunk.text]=[chunk.text, chunk.root.text, chunk.root.dep_,chunk.root.head.text]
        return samplechunk
       
    def GetAvgDepth(self):
        
        def getdepth(token):
             # if right child exist search right child
            if not list(token.children):
                return 0
            else:
                leftmax= 0 if not list(token.lefts) else max([getdepth(i)  for i in token.lefts])
            # if left child exist search leftchild
                rightmax=0  if not list(token.rights) else max([getdepth(i)  for i in token.rights])
                return max(leftmax,rightmax)+1
            # if not child return 0
            # return the max number
        count=0
        mydepth=dict()
        for token in self.doc:
            if token.dep_=="ROOT":
                # iterating each dep and get the depth
                count=count+1
                mydepth[count]=getdepth(token)
        return(sum(mydepth.values())/count)
    def GetAvgSenlen(self):
        sentences = [sent.string.strip() for sent in self.doc.sents]
        return len(self.doc)/len(sentences)
    
    def Getall(self):
        return pd.DataFrame([[self.wordnum,self.trivnum,len(self.GetEntityDF()),len(self.GetWordChunkDF()),self.GetAvgDepth(),self.GetAvgSenlen()]],\
                            columns=["Total words","Trivial words","entities","wordchunk","AvgDepth","AvgLen"],index=[self.docname]) 
    
        
        

In [81]:
prossessor=FileProcessor("sample",remove_stop_words(doc_complete[0])) # build a file processor

In [82]:
prossessor.Getall()

,Total words,Trivial words,entities,wordchunk,AvgDepth,AvgLen
sample,475,227,50,94,5.466667,31.666667


In [83]:
prossessor.GetEntityDF()

,text,start_char,end_char,label
US,US,3112,3114,GPE
Zhao Huanxin,Zhao Huanxin,83,95,PERSON
China Daily Updated,China Daily Updated,96,115,ORG
2018,2018,116,120,DATE
18 08,18 08,124,129,DATE
46,46,130,132,DATE
Donald Trump,Donald Trump,233,245,PERSON
Chinese,Chinese,576,583,NORP
April 5,April 5,465,472,DATE
Trump,Trump,3092,3097,NORP
